<a href="https://colab.research.google.com/github/sargunasaravanan/GenAI-Workouts/blob/main/LORA_FineTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install peft

In [ ]:
model_name = "bigscience/bloomz-560m"

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map=device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [ ]:
def get_output(model, inputs, max_new_tokens=100):
    output = model.generate(input_ids=inputs["input_ids"],
                   attention_mask = inputs["attention_mask"],
                   max_new_tokens=max_new_tokens)
    return tokenizer.decode(output[0], skip_special_token=True)

In [ ]:
prompt = "i want you at act as a motivational coach"

In [ ]:
batch = tokenizer(prompt, return_tensors='pt')
batch

{'input_ids': tensor([[   76,  4026,  1152,   919,  1769,   661,   267, 22675,  3383, 67604]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
get_output(model, batch.to(device))

'i want you at act as a motivational coach</s>'

In [ ]:
dataset = "fka/awesome-chatgpt-prompts"

In [ ]:
from datasets import load_dataset

In [ ]:
data = load_dataset(dataset)

README.md:   0%|          | 0.00/339 [00:00<?, ?B/s]

prompts.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/203 [00:00<?, ? examples/s]

In [ ]:
data["train"]

Dataset({
    features: ['act', 'prompt'],
    num_rows: 203
})

In [ ]:
train_sample = data["train"].select(range(50))

In [ ]:
# import datasets
# help(datasets)

In [ ]:
train_sample = train_sample.remove_columns("act")

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize
count = []
for i in range(len(train_sample)):
  count.append(len(word_tokenize(train_sample[i]["prompt"])))

In [ ]:
min(count)

68

In [ ]:
max(count)

158

In [ ]:
train_sample = train_sample.map(lambda x: tokenizer(x["prompt"], max_length=175, truncation=True), batched=True)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
train_sample

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 50
})

In [ ]:
import peft

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType, PeftModel

In [ ]:
lora_config = LoraConfig(r=8,
           lora_alpha=1,
           target_modules=["query_key_value"],
           lora_dropout=0.05,
           bias="lora_only",
            task_type=TaskType.CAUSAL_LM)

In [ ]:
peft_model = get_peft_model(model, lora_config)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
peft_model.print_trainable_parameters()

trainable params: 860,160 || all params: 560,001,024 || trainable%: 0.1536


In [ ]:
import os
output_dir = os.path.join("./", "peft_outputs")

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
args = TrainingArguments(output_dir=output_dir,
                  learning_rate=0.005,
                  num_train_epochs=2,
                  auto_find_batch_size=True)

In [ ]:
import transformers
trainer = Trainer(model=peft_model,
        train_dataset=train_sample,
        args=args,
                  data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False))

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=14, training_loss=2.8425134931291853, metrics={'train_runtime': 12.5805, 'train_samples_per_second': 7.949, 'train_steps_per_second': 1.113, 'total_flos': 21899574263808.0, 'train_loss': 2.8425134931291853, 'epoch': 2.0})

In [ ]:
trainer.model.save_pretrained("lora_model")

In [ ]:
loaded_model = PeftModel.from_pretrained(model,"lora_model", is_trainable=False)

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
loaded_model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): BloomForCausalLM(
      (transformer): BloomModel(
        (word_embeddings): Embedding(250880, 1024)
        (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (h): ModuleList(
          (0-23): 24 x BloomBlock(
            (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (self_attention): BloomAttention(
              (query_key_value): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=3072, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=3072, bias=False)
                )
                (lora_e

In [ ]:
prompt

'i want you at act as a motivational coach'

In [ ]:
i1 =tokenizer(prompt, return_tensors="pt")
i1

{'input_ids': tensor([[   76,  4026,  1152,   919,  1769,   661,   267, 22675,  3383, 67604]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
get_output(loaded_model, i1.to(device), max_new_tokens=100)

'i want you at act as a motivational coach. I want you to help people overcome their fears and overcome their insecurities. I want you to help people overcome their fears and overcome their insecurities. I want you to help people overcome their fears and overcome their insecurities. I want you to help people overcome their fears and overcome their insecurities. I want you to help people overcome their fears and overcome their insecurities. I want you to help people overcome their fears and overcome their insecurities. I want you'

In [ ]:
volume-rows
dimension-columns

(175billion), 1lakh

In [ ]:
small dataset

fine tuning- train(data)--base model knowledge-small-memorizing)-catastrophic issue